In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
from implementations import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
# We Separate the dataset
from implementations import *

tX_list, ids_list, y_list = separate_dataset(tX, ids, y)

## Do your thing crazy machine learning thing here :) ...

In [4]:
lambda_ = 1e-3
# Ridge regression on separated datset with data augmentation
mat_tX_list = build_poly_separated(tX_list, 2, log=True)
weights_list, loss_list = separated_train(mat_tX_list, y_list, ridge_regression, lambda_)

In [5]:
loss_list

[0.2519574420849784, 0.3390375170050995, 0.322627983620031, 0.3320368398490934]

## Generate predictions and save ouput in csv format for submission:

In [6]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [7]:
# Data augmentation and separated dataset
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission

tX_test_list, ids_test_list = separate_dataset(tX_test, ids_test)   #separate dateset
mat_tX_test_list = build_poly_separated(tX_test_list, 2)            #data augmentation
y_pred_list = separated_eval(weights_list, mat_tX_test_list)        #prediction

y_pred = np.concatenate((y_pred_list[0], y_pred_list[1], y_pred_list[2], y_pred_list[3]))
ids_test_sub = np.concatenate((ids_test_list[0], ids_test_list[1], ids_test_list[2], ids_test_list[3]))

create_csv_submission(ids_test_sub, y_pred, OUTPUT_PATH)

ValueError: shapes (227458,152) and (159,) not aligned: 152 (dim 1) != 159 (dim 0)